<a href="https://colab.research.google.com/github/OlegBEZb/NLP_advanced_course/blob/master/HW1/NLP%20advance%20course.%20HW3.%20Distributive%20semantic.%20Embedings_%2B_Bidirectional_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Embeddings to check

* Word Embeddings (word2vec, GloVe, etc.)
* Starspace and other similarity learning embeddings
* char-gram embeddings (bpemb, fasttext etc.)
* doc and sentence embeddings (doc2vec, sent2vec etc.)
* specific context models (BERT, ELMo)
* Advanced: Poincare Embeddings concept

# Libs import

In [2]:
import os, math, operator, csv, random, pickle, re, sys
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import gc
from collections import Counter

import keras
from keras.layers import MaxPooling1D, BatchNormalization, Permute, Lambda, \
Activation, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dense, \
Embedding, Dropout, Input, CuDNNGRU, merge, CuDNNLSTM, Flatten, \
TimeDistributed, concatenate, SpatialDropout1D, Bidirectional
from nltk.util import pad_sequence
from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

import gensim
from gensim.models import KeyedVectors

from keras import backend as K

import nltk
nltk.download('stopwords');
from nltk.tokenize import TweetTokenizer

from nltk.corpus import stopwords

! pip install Unidecode;
from unidecode import unidecode
#! pip install pyspellchecker;
#from spellchecker import SpellChecker

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from keras import backend as K

import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, CuDNNGRU, Conv1D
from keras.preprocessing import text
from keras.callbacks import LearningRateScheduler, Callback
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 245kB 2.7MB/s 
1.15.0


# Check GPU

In [3]:
if tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
):
    print(tf.test.gpu_device_name())

/device:GPU:0


# Global vars

In [0]:
COLAB = True

BATCH_SIZE = 512
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
GLOBAL_EPOCHS = 2
EPOCHS = 5

# Authorization on Google drive and configurings paths

In [5]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    homework_folder = os.path.join('/content/drive/My Drive', 'Advanced NLP/Homework 1: Classical classification task like Kaggle Toxic or Quora')
    data_folder = os.path.join(homework_folder, 'Toxic data')
    embeddings_folder = os.path.join(homework_folder, 'embeddings')
    print('data found:', os.listdir(data_folder))
else: # in case of launching in Kaggle kernel
    data_folder = '../input/jigsaw-toxic-comment-classification-challenge/'
    embeddings_folder = '../input/glove-global-vectors-for-word-representation'
    print('data found:', os.listdir(data_folder))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
data found: ['test.csv', 'test_labels.csv', 'train.csv', 'sample_submission.csv', 'submission.csv', 'submission_emb_sklearn.csv', 'submission_bidirectional_GRU.csv']


In [0]:
train_filepath = os.path.join(data_folder,"train.csv")
test_filepath = os.path.join(data_folder,"test.csv")
test_labels_filepath = os.path.join(data_folder,"test_labels.csv")

#path to a submission
submission_path = os.path.join(data_folder,"sample_submission.csv")

# Preprocessing

## Dicts and lists of useful words and transformations

In [0]:
#List of some words that often appear in toxic comments
#Sorry about the level of toxicity in it!
toxic_words = ["poop", "crap", "prick", "twat", "wikipedia", "wiki", "hahahahaha", "lol", "bastard", "sluts", "slut", "douchebag", "douche", "blowjob", "nigga", "dumb", "jerk", "wanker", "wank", "penis", "motherfucker", "fucker", "fuk", "fucking", "fucked", "fuck", "bullshit", "shit", "stupid", "bitches", "bitch", "suck", "cunt", "dick", "cocks", "cock", "die", "kill", "gay", "jewish", "jews", "jew", "niggers", "nigger", "faggot", "fag", "asshole"]
astericks_words = [('mother****ers', 'motherfuckers'), ('motherf*cking', 'motherfucking'), ('mother****er', 'motherfucker'), ('motherf*cker', 'motherfucker'), ('bullsh*t', 'bullshit'), ('f**cking', 'fucking'), ('f*ucking', 'fucking'), ('fu*cking', 'fucking'), ('****ing', 'fucking'), ('a**hole', 'asshole'), ('assh*le', 'asshole'), ('f******', 'fucking'), ('f*****g', 'fucking'), ('f***ing', 'fucking'), ('f**king', 'fucking'), ('f*cking', 'fucking'), ('fu**ing', 'fucking'), ('fu*king', 'fucking'), ('fuc*ers', 'fuckers'), ('f*****', 'fucking'), ('f***ed', 'fucked'), ('f**ker', 'fucker'), ('f*cked', 'fucked'), ('f*cker', 'fucker'), ('f*ckin', 'fucking'), ('fu*ker', 'fucker'), ('fuc**n', 'fucking'), ('ni**as', 'niggas'), ('b**ch', 'bitch'), ('b*tch', 'bitch'), ('c*unt', 'cunt'), ('f**ks', 'fucks'), ('f*ing', 'fucking'), ('ni**a', 'nigga'), ('c*ck', 'cock'), ('c*nt', 'cunt'), ('cr*p', 'crap'), ('d*ck', 'dick'), ('f***', 'fuck'), ('f**k', 'fuck'), ('f*ck', 'fuck'), ('fc*k', 'fuck'), ('fu**', 'fuck'), ('fu*k', 'fuck'), ('s***', 'shit'), ('s**t', 'shit'), ('sh**', 'shit'), ('sh*t', 'shit'), ('tw*t', 'twat')]
fasttext_misspelings = {"'n'balls": 'balls', "-nazi's": 'nazis', 'adminabuse': 'admin abuse', "admins's": 'admins', 'arsewipe': 'arse wipe', 'assfack': 'asshole', 'assholifity': 'asshole', 'assholivity': 'asshole', 'asshoul': 'asshole', 'asssholeee': 'asshole', 'belizeans': 'mexicans', "blowing's": 'blowing', 'bolivians': 'mexicans', 'celtofascists': 'fascists', 'censorshipmeisters': 'censor', 'chileans': 'mexicans', 'clerofascist': 'fascist', 'cowcrap': 'crap', 'crapity': 'crap', "d'idiots": 'idiots', 'deminazi': 'nazi', 'dftt': "don't feed the troll", 'dildohs': 'dildo', 'dramawhores': 'drama whores', 'edophiles': 'pedophiles', 'eurocommunist': 'communist', 'faggotkike': 'faggot', 'fantard': 'retard', 'fascismnazism': 'fascism', 'fascistisized': 'fascist', 'favremother': 'mother', 'fuxxxin': 'fucking', "g'damn": 'goddamn', 'harassmentat': 'harassment', 'harrasingme': 'harassing me', 'herfuc': 'motherfucker', 'hilterism': 'fascism', 'hitlerians': 'nazis', 'hitlerites': 'nazis', 'hubrises': 'pricks', 'idiotizing': 'idiotic', 'inadvandals': 'vandals', "jackass's": 'jackass', 'jiggabo': 'nigga', 'jizzballs': 'jizz balls', 'jmbass': 'dumbass', 'lejittament': 'legitimate', "m'igger": 'nigger', "m'iggers": 'niggers', 'motherfacking': 'motherfucker', 'motherfuckenkiwi': 'motherfucker', 'muthafuggas': 'niggas', 'nazisms': 'nazis', 'netsnipenigger': 'nigger', 'niggercock': 'nigger', 'niggerspic': 'nigger', 'nignog': 'nigga', 'niqqass': 'niggas', "non-nazi's": 'not a nazi', 'panamanians': 'mexicans', 'pedidiots': 'idiots', 'picohitlers': 'hitler', 'pidiots': 'idiots', 'poopia': 'poop', 'poopsies': 'poop', 'presumingly': 'obviously', 'propagandaanddisinformation': 'propaganda and disinformation', 'propagandaministerium': 'propaganda', 'puertoricans': 'mexicans', 'puertorricans': 'mexicans', 'pussiest': 'pussies', 'pussyitis': 'pussy', 'rayaridiculous': 'ridiculous', 'redfascists': 'fascists', 'retardzzzuuufff': 'retard', "revertin'im": 'reverting', 'scumstreona': 'scums', 'southamericans': 'mexicans', 'strasserism': 'fascism', 'stuptarded': 'retarded', "t'nonsense": 'nonsense', "threatt's": 'threat', 'titoists': 'communists', 'twatbags': 'douchebags', 'youbollocks': 'you bollocks'}
acronym_words = {"btw":"by the way", "yo": "you", "u": "you", "r": "are", "ur": "your", "ima": "i am going to", "imma": "i am going to", "i'ma":"i am going to", "cos":"because", "coz":"because", "stfu": "shut the fuck up", "wat": "what"}
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

english_stopwords = set(stopwords.words('english'))

#spell_checker = SpellChecker()

cont_patterns = [
    (r'(W|w)on\'t', r'will not'),
    (r'(C|c)an\'t', r'can not'),
    (r'(I|i)\'m', r'i am'),
    (r'(A|a)in\'t', r'is not'),
    (r'(\w+)\'ll', r'\g<1> will'),
    (r'(\w+)n\'t', r'\g<1> not'),
    (r'(\w+)\'ve', r'\g<1> have'),
    (r'(\w+)\'s', r'\g<1> is'),
    (r'(\w+)\'re', r'\g<1> are'),
    (r'(\w+)\'d', r'\g<1> would'),
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

#We will filter all characters except alphabet characters and some punctuation
valid_characters = " " + "@$" + "'!?-" + "abcdefghijklmnopqrstuvwxyz" + "abcdefghijklmnopqrstuvwxyz".upper()
valid_set = set(x for x in valid_characters)

## Funcs for processing

In [0]:
def replace_url(word):
    if "http://" in word or "www." in word or "https://" in word or "wikipedia.org" in word:
        return ""
    return word


def word_tokenize(sentence, tokenizer):
    sentence = sentence.replace("$", "s")
    sentence = sentence.replace("@", "a")    
    sentence = sentence.replace("!", " ! ")
    sentence = sentence.replace("?", " ? ")

    return tokenizer.tokenize(sentence)


def split_toxic_and_normal_parts(word, toxic_words):
    if word == "":
        return ""
    
    lower = word.lower()
    for toxic_word in toxic_words:
        start = lower.find(toxic_word)
        if start >= 0:
            end = start + len(toxic_word)
            result = " ".join([word[0:start], word[start:end], split_toxic_and_normal_parts(word[end:], toxic_words)])
            return result.replace("  ", " ").strip()
    return word


def normalize_by_dictionary(normalized_word, dictionary):
    result = []
    for word in normalized_word.split():
        if word == word.upper():
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()].upper())
            else:
                result.append(word)
        else:
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()])
            else:
                result.append(word)
    
    return " ".join(result)


def normalize_comment(comment, tokenizer, max_comment_length):
    comment = unidecode(comment)
    comment = comment[:max_comment_length]

    normalized_words = []
    
    # ('mother****ers', 'motherfuckers')
    # for w in astericks_words:
    #     if w[0] in comment:
    #         comment = comment.replace(w[0], w[1])
    #     if w[0].upper() in comment:
    #         comment = comment.replace(w[0].upper(), w[1].upper())
    
    for word in word_tokenize(comment, tokenizer):
        if word in english_stopwords: continue

        # # '(W|w)on\'t', r'will not'
        # for (pattern, repl) in patterns:
        #    word = re.sub(pattern, repl, word)

        if word == "." or word == ",":
            normalized_words.append(word)
            continue
        
        # drop url parts from links
        # word = replace_url(word)

        # replace single dots to whitespaces
        if word.count(".") == 1:
            word = word.replace(".", " ")

        # leave only legalized symbols
        filtered_word = "".join([x for x in word if x in valid_set])
                    
        #Kind of hack: for every word check if it has a toxic word as a part of it
        #If so, split this word by swear and non-swear part.
        #normalized_word = split_toxic_and_normal_parts(filtered_word, toxic_words)
        normalized_word = filtered_word

#         normalized_word = normalize_by_dictionary(normalized_word, hyphens_dict)
#         normalized_word = normalize_by_dictionary(normalized_word, merged_dict)
        
        # check misspellings
        # temp = []
        # for word in normalized_word.split():
        #   temp.append(spell_checker.correction(word))
        # normalized_word = " ".join(temp)
          
        # normalized_word = normalize_by_dictionary(normalized_word, fasttext_misspelings)
        normalized_word = normalize_by_dictionary(normalized_word, acronym_words)

        normalized_words.append(normalized_word)
        
    normalized_comment = " ".join(normalized_words)
    
    result = []
    for word in normalized_comment.split():
        # if word is upper
        if word.upper() == word:
            result.append(word)
        else:
            result.append(word.lower())
    
    #apparently, people on wikipedia love to talk about sockpuppets :-)
    result = " ".join(result)
    if "sock puppet" in result:
        result = result.replace("sock puppet", "sockpuppet")
    
    if "SOCK PUPPET" in result:
        result = result.replace("SOCK PUPPET", "SOCKPUPPET")
    
    return result


def preprocess_text(df, preprocessing_func, tokenizer, max_comment_length):
  text_ndarray = df.fillna('_na').values
  np_preprocessing_func = np.vectorize(preprocessing_func)

  preprocessed_text = np_preprocessing_func(text_ndarray, tokenizer, max_comment_length)

  print('Gained shape:', preprocessed_text.shape)
  return preprocessed_text


def tokenize_and_pad(text, tokenizer):

  tokenized_text = tokenizer.texts_to_sequences(text)
  padded_text = pad_sequences(tokenized_text, maxlen=MAX_LEN, dtype='int', value=0)

  print('Gained shape:', padded_text.shape)
  return padded_text

## Loading frames and processing them

In [0]:
TEXT_COLUMN = 'comment_text'
TARGET_COLS = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

MAX_LEN = 220
max_tokens = 20000
max_comment_length = 20000 #We are going to truncate a comment if its length > threshold

In [14]:
%%time

train_df = pd.read_csv(train_filepath)
train_labels = train_df[TARGET_COLS].values

CPU times: user 653 ms, sys: 70.9 ms, total: 724 ms
Wall time: 767 ms


In [15]:
%%time

tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)

preprocessed_train = preprocess_text(train_df[TEXT_COLUMN], 
                                     normalize_comment, 
                                     tknzr,
                                     max_comment_length)
# # preprocessed_train = train_df[TEXT_COLUMN].fillna("CVxTz").values
# print(preprocessed_train[:3])
# preprocessed_train = train_df[TEXT_COLUMN]

del train_df

Gained shape: (159571,)
CPU times: user 56.3 s, sys: 3.02 s, total: 59.3 s
Wall time: 59.3 s


In [0]:
%%time

tokenizer = Tokenizer(num_words=max_tokens)
tokenizer.fit_on_texts(list(preprocessed_train))

tokenized_train = tokenize_and_pad(preprocessed_train, tokenizer)
# list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
# list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
#print(tokenized_train[:3])

del preprocessed_train

In [16]:
%%time

test_df = pd.read_csv(test_filepath)

preprocessed_test = preprocess_text(test_df[TEXT_COLUMN], 
                                    normalize_comment, 
                                    tknzr,
                                    max_comment_length)
# preprocessed_test = test_df[TEXT_COLUMN].fillna("CVxTz").values
# preprocessed_test = test_df[TEXT_COLUMN]



#del preprocessed_test
del test_df
gc.collect()

Gained shape: (153164,)
CPU times: user 52.5 s, sys: 260 ms, total: 52.7 s
Wall time: 53.2 s


In [0]:
tokenized_test = tokenize_and_pad(preprocessed_test, tokenizer)

del preprocessed_test
gc.collect()

# Training model

## Preparing features

In [0]:
embedding_len = 1536

word_index = tokenizer.word_index
print('Found %s unique tokens.' %len(word_index))

#del tokenizer
gc.collect()

num_words = min(max_tokens, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_len)) # zeroth row for UNK
print('embedding matrix shape', embedding_matrix.shape)

Found 184924 unique tokens.
embedding matrix shape (20000, 1536)


### GloVe

In [0]:
# embedding_len = 200
# words = pd.read_csv(os.path.join(embeddings_folder, "glove.twitter.27B.{}d.txt".format(embedding_len)), 
#                     sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

# embedding_len = 300
# words = pd.read_csv(os.path.join(embeddings_folder, "glove.840B.300d.txt"), 
#                     sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)


# vocab_words = words[words.index.isin(list(word_index.keys()))]
# print('vocab_words.shape', vocab_words.shape)

# del words
# gc.collect()

In [0]:
# def get_vec(word, words_df):
#     return words_df.loc[word].values



# for word, i in word_index.items():
#   if i >= max_tokens:
#     continue
#   try:
#     embedding_vector = get_vec(word, vocab_words)
#   except:
#     continue
#   if embedding_vector is not None:
#     # words not found in embedding index will be all-zeros.
#     embedding_matrix[i] = embedding_vector

# # %%time
# # embedding_matrix = np.load('../input/embedding-2/embedding_matrix_big.npy')
    
# print('embedding_matrix.shape', embedding_matrix.shape)

### StarSpace

In [0]:
# Starspace attempt

# from sklearn.feature_extraction.text import CountVectorizer
# count_vectorizer = CountVectorizer(min_df=0, max_df=0.99, max_features=10000)
# X_train = count_vectorizer.fit_transform(article_contents.main_content.iloc[0:train_row])
# X_train = count_vectorizer.inverse_transform(X_train)
# label_prefix = '__label__'
# with open("train_starspace.txt", 'w+') as file:
#     for i in range(10):
#         file.write(' '.join(preprocessed_train[i].split(' ')) + ' ' + label_prefix + train_labels[i])
#         file.write('\n')
# file.close()

# The result file will look like this (all separeted by space, and label will have prefix __label__)
# how are you ... __label__b
# this is just an example ... __label__c

### FastText

In [0]:
!pip install fasttext

In [0]:
# import codecs

# #load embeddings
# print('loading word embeddings...')
# embeddings_index = {}
# f = codecs.open(os.path.join(embeddings_folder, 'crawl-300d-2M-subword.vec'), encoding='utf-8')
# for line in tqdm(f, mininterval=5):
#     values = line.rstrip().rsplit(' ')
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()
# print('found %s word vectors' % len(embeddings_index))
# print('preparing embedding matrix...')
# words_not_found = []


# for word, i in word_index.items():
#     if i >= max_tokens:
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if (embedding_vector is not None) and len(embedding_vector) > 0:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
#     else:
#         words_not_found.append(word)
# print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


# words_not_found[:10]

### BPEmb

In [0]:
# !pip install bpemb;
# from bpemb import BPEmb
# # load English BPEmb model with default vocabulary size of max_tokens
# bpemb_en = BPEmb(lang="en", dim=embedding_len, vs=max_tokens)
# bpemb_en.vectors.shape

In [0]:
# # mean vector of word parts
# def get_vec(word):
#     vec = bpemb_en.embed(word)
#     return vec.mean(axis=0)

In [0]:
# %%time

# for word, i in word_index.items():
#   if i >= max_tokens:
#     continue
#   try:
#     embedding_vector = get_vec(word)
#   except:
#     continue
#   if embedding_vector is not None:
#     # words not found in embedding index will be all-zeros.
#     embedding_matrix[i] = embedding_vector

# print('embedding_matrix.shape', embedding_matrix.shape)

embedding_matrix.shape (20000, 300)
CPU times: user 504 ms, sys: 18.1 ms, total: 522 ms
Wall time: 520 ms


### Flair

In [0]:
! pip install tiny-tokenizer
! pip install flair

import flair
from flair.data import Sentence
import torch

  Created wheel for tiny-tokenizer: filename=tiny_tokenizer-3.1.0-cp36-none-any.whl size=10550 sha256=aa15269699b787314d1c790861bba985df9096ce9ad75c95d32baf09b8b5ec6d
  Stored in directory: /root/.cache/pip/wheels/d1/c8/36/334497a689fab90128232e86b5829b800dd271a3d5d5959c53
Successfully built tiny-tokenizer
     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 450kB 48.1MB/s 
     |████████████████████████████████| 1.0MB 51.4MB/s 
     |████████████████████████████████| 778kB 35.3MB/s 
     |████████████████████████████████| 798kB 43.1MB/s 
     |████████████████████████████████| 1.0MB 35.7MB/s 
     |████████████████████████████████| 870kB 47.0MB/s 
     |████████████████████████████████| 70.7MB 44kB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 21.5MB 158kB/s 
     |████████████████████████████████| 3.8MB 30.9MB/s 
     |████████████████████████████████| 348kB 44.2MB/s 
  Created wheel for lang

#### ELMo

In [0]:
! pip install allennlp

In [0]:
from flair.embeddings import ELMoEmbeddings

# init embedding
embedding = ELMoEmbeddings('medium')

# create a sentence #
sentence = Sentence('test sent')
# embed words in sentence #
embedding.embed(sentence)
embedding_len = sentence[0].embedding.size()[0]
print('embedding len', embedding_len)


def get_comment_emb(comment):
  sentence = Sentence(comment)
  embedding.embed(sentence)
  emb_list = []
  for word in sentence:
    emb_list.append(word.embedding)
  return emb_list

get_comment_emb_vec = np.vectorize(get_comment_emb)

embedding len 1536


In [0]:
for word, i in tqdm(word_index.items(), total=len(word_index)):
    if i >= max_tokens:
        continue
    try:
        word_sent = Sentence(word)
        embedding.embed(word_sent)
        embedding_vector = word_sent[0].embedding.cpu().detach().numpy()
        if (embedding_vector is not None) and len(embedding_vector) > 0:
           embedding_matrix[i] = embedding_vector
    except KeyError:
        pass
        #embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

100%|██████████| 184924/184924 [03:02<00:00, 1014.44it/s]


In [0]:
del embedding
gc.collect()

648

In [0]:
# # creating a tensor for storing sentence embeddings #
# s = torch.zeros(0, embedding_len)

# # iterating Sentence (tqdm tracks progress) #
# for tweet in tqdm(preprocessed_train[10]):   
#   # empty tensor for words #
#   w = torch.zeros(0, embedding_len)   
#   sentence = Sentence(tweet)
#   embedding.embed(sentence)
#   # for every word #
#   for token in sentence:
#     # storing word Embeddings of each word in a sentence #
#     w = torch.cat((w,token.embedding.view(-1, embedding_len)),0)
#   # storing sentence Embeddings (mean of embeddings of all words)   #
#   s = torch.cat((s, w.mean(dim = 0).view(-1, embedding_len)),0)

#### RoBERTa

In [0]:
from flair.embeddings import RoBERTaEmbeddings

In [0]:
# init embedding
embedding = RoBERTaEmbeddings('roberta-base')

# create a sentence #
sentence = Sentence('test sent')
# embed words in sentence #
embedding.embed(sentence)
embedding_len = sentence[0].embedding.size()[0]
print('embedding len', embedding_len)

embedding len 768


In [0]:
for word, i in tqdm(word_index.items(), total=len(word_index)):
    if i >= max_tokens:
        continue
    try:
        word_sent = Sentence(word)
        embedding.embed(word_sent)
        embedding_vector = word_sent[0].embedding.cpu().detach().numpy()
        if (embedding_vector is not None) and len(embedding_vector) > 0:
           embedding_matrix[i] = embedding_vector
    except KeyError:
        pass
        #embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del embedding
gc.collect()

100%|██████████| 184924/184924 [04:29<00:00, 685.45it/s]


648

#### RoBERTa + ELMo

In [0]:
! pip install allennlp
from flair.embeddings import StackedEmbeddings, RoBERTaEmbeddings, ELMoEmbeddings

In [0]:
elmo_embeddings = ELMoEmbeddings('small')
roberta_embeddings = RoBERTaEmbeddings('roberta-base')
stacked_embeddings = StackedEmbeddings([
                                        elmo_embeddings,
                                        roberta_embeddings,
                                       ])

# create a sentence #
sentence = Sentence('test sent')
# embed words in sentence #
stacked_embeddings.embed(sentence)
embedding_len = sentence[0].embedding.size()[0]
print('embedding len', embedding_len)

100%|██████████| 54402456/54402456 [00:02<00:00, 20212502.71B/s]


embedding len 1536


In [0]:
for word, i in tqdm(word_index.items(), total=len(word_index)):
    if i >= max_tokens:
        continue
    try:
        word_sent = Sentence(word)
        stacked_embeddings.embed(word_sent)
        embedding_vector = word_sent[0].embedding.cpu().detach().numpy()
        if (embedding_vector is not None) and len(embedding_vector) > 0:
           embedding_matrix[i] = embedding_vector
    except KeyError:
        pass
        #embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

100%|██████████| 184924/184924 [11:48<00:00, 260.89it/s]


In [0]:
del stacked_embeddings
gc.collect()

549

### BERT fine-tuned

In [0]:
import tensorflow_hub as hub

In [18]:
#Installing BERT module
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.1MB/s 


In [19]:
#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [20]:
OUTPUT_DIR = os.path.join(homework_folder, 'bert_output')
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /content/drive/My Drive/Advanced NLP/Homework 1: Classical classification task like Kaggle Toxic or Quora/bert_output *****


In [21]:
preprocessed_train_with_labels = pd.concat([pd.DataFrame(preprocessed_train, columns=['preprocessed_text']),
                                            pd.DataFrame(train_labels, columns=TARGET_COLS)], axis=1)
preprocessed_train_with_labels.head(2)

,preprocessed_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation why edits made username hardcore m...,0,0,0,0,0,0
1,d'aww ! he matches background colour i'm seemi...,0,0,0,0,0,0


In [0]:
def create_examples(df, labels_available=True):
  examples = []
  for (i, row) in enumerate(df.values):
    guid = None#row[0]
    text_a = row[0]
    if labels_available:
      labels = tuple(row[1:])
    else:
      labels = tuple([0, 0, 0, 0, 0, 0])
    examples.append(bert.run_classifier.InputExample(guid=guid, text_a = text_a, 
                                                     label = labels))
  return examples

In [36]:
%%time

input_examples = create_examples(preprocessed_train_with_labels, labels_available=True)

CPU times: user 878 ms, sys: 22 ms, total: 900 ms
Wall time: 901 ms


In [37]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

bert_tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [38]:
%%time

train_input_examples, valid_input_examples = train_test_split(input_examples, test_size = 0.1, shuffle=True)

CPU times: user 58.8 ms, sys: 991 µs, total: 59.8 ms
Wall time: 59.2 ms


In [39]:
len(train_input_examples), type(train_input_examples[0])

(143613, bert.run_classifier.InputExample)

In [40]:
train_input_examples[0].label

(1, 0, 1, 0, 0, 0)

In [0]:
MAX_SEQ_LENGTH = MAX_LEN

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_input_examples, TARGET_COLS, MAX_SEQ_LENGTH, bert_tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(valid_input_examples, TARGET_COLS, MAX_SEQ_LENGTH, bert_tokenizer)

## Build and fit the model

In [0]:
x_train, x_valid, y_train, y_valid = train_test_split(tokenized_train, 
                                                      train_labels, 
                                                      test_size = 0.1,
                                                      shuffle=True)

print('x_train shape:', x_train.shape)
print('x_valid shape:', x_valid.shape)
print('y_train shape:', y_train.shape)
print('y_valid shape:', y_valid.shape)

del tokenized_train
gc.collect()

x_train shape: (143613, 220)
x_valid shape: (15958, 220)
y_train shape: (143613, 6)
y_valid shape: (15958, 6)


0

In [0]:
def build_model(embedding_matrix):
    words = Input(shape=(MAX_LEN,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    #x = Input(batch_shape=(batch_size, max_comment_length, embedding_len))
    
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(len(TARGET_COLS), activation='sigmoid')(hidden)
    
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

In [0]:
# class RocAucEvaluation(Callback):
#     def __init__(self, validation_data=(), interval=1):
#         super(Callback, self).__init__()

#         self.interval = interval
#         self.X_val, self.y_val = validation_data

#     def on_epoch_end(self, epoch, logs={}):
#         if epoch % self.interval == 0:
#             y_pred = self.model.predict(self.X_val, verbose=0)
#             score = 0
#             for i in range(6):
#              score += roc_auc_score(self.y_val[:,i], y_pred[:,i])/6.
#             print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

# RocAuc_val = RocAucEvaluation(validation_data=(x_valid, y_valid), interval = 1)

class RocAucEarlyStopping(Callback):
    """Callback for early stopping based on roc auc on the validation set"""

    def __init__(self, validation_data, patience=0, digits=3):
        super().__init__()
        self.X_val, self.y_val = validation_data
        self.best = 0
        self.patience = patience
        self.digits = digits
        self.current_patience = patience

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = 0
        for i in range(6):
          score += roc_auc_score(self.y_val[:,i], y_pred[:,i])/6.

        score = round(score, self.digits)
        print("ROC-AUC - epoch: {:d} - score: {}\n".format(epoch+1, score))

        # check digits
        if np.greater(score, self.best):
            self.best = score
            self.current_patience = self.patience
        else:
            print('\nbest:{}\ncurrent:{}'.format(self.best, score))
            self.current_patience -= 1
            if self.current_patience < 0:
              self.model.stop_training = True
              print('Early stopping due to lower roc auc')
              self.current_patience = self.patience
            else:
              print('{} patience remained'.format(self.current_patience))

RocAuc_ES = RocAucEarlyStopping(validation_data=(x_valid, y_valid), patience=1, digits=4)

In [0]:
%%time

SEEDS = 2
pred = 0

for ii in tqdm(range(SEEDS), total=SEEDS):

    model = build_model(embedding_matrix)
    for global_epoch in range(GLOBAL_EPOCHS):
        print('\nglobal_epoch', global_epoch)


        model.fit(
                    x_train,
                    y_train,
                    validation_data = (x_valid, y_valid),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=[
                        LearningRateScheduler(lambda _: 1e-3 * (0.5 ** global_epoch)),
                        #RocAuc_val
                        RocAuc_ES
                    ]
                )
        print('fitted')    

    pred += model.predict(tokenized_test, batch_size = 1024, verbose = 1)/SEEDS
    np.save('pred', pred)
    model.save_weights('model_weights_'+str(ii)+'.h5')
    os.system('gzip '+'model_weights_'+str(ii)+'.h5')

  0%|          | 0/2 [00:00<?, ?it/s]











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

global_epoch 0


Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 87s 605us/step - loss: 0.0700 - val_loss: 0.0497
ROC-AUC - epoch: 1 - score: 0.9787

Epoch 2/5
143613/143613 [==============================] - 82s 569us/step - loss: 0.0473 - val_loss: 0.0446
ROC-AUC - epoch: 2 - score: 0.9856

Epoch 3/5
143613/143613 [==============================] - 82s 569us/step - loss: 0.0429 - val_loss: 0.0445
ROC-AUC - epoch: 3 - score: 0.9868

Epoch 4/5
143613/143613 [==============================] - 82s 568us/step - loss: 0.0402 - val_loss: 0.0428
ROC-AUC - epoch: 4 - score: 0.9879

Epoch 5/5
143613/143613 [==============================] - 82s 569us/step - loss: 0.0382 - val_loss: 0.0447
ROC-AUC - epoch: 5 - score

 50%|█████     | 1/2 [11:51<11:51, 711.77s/it]


global_epoch 0
Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 82s 570us/step - loss: 0.0693 - val_loss: 0.0518
ROC-AUC - epoch: 1 - score: 0.98


best:0.9881
current:0.98
0 patience remained
Epoch 2/5
143613/143613 [==============================] - 81s 567us/step - loss: 0.0469 - val_loss: 0.0469
ROC-AUC - epoch: 2 - score: 0.9854


best:0.9881
current:0.9854
Early stopping due to lower roc auc
fitted

global_epoch 1
Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 81s 567us/step - loss: 0.0417 - val_loss: 0.0426
ROC-AUC - epoch: 1 - score: 0.9873


best:0.9881
current:0.9873
0 patience remained
Epoch 2/5
143613/143613 [==============================] - 81s 566us/step - loss: 0.0395 - val_loss: 0.0427
ROC-AUC - epoch: 2 - score: 0.9878


best:0.9881
current:0.9878
Early stopping due to lower roc auc
fitted
153164/153164 [==============================] - 33s 213

100%|██████████| 2/2 [18:48<00:00, 623.22s/it]

CPU times: user 10min 36s, sys: 4min, total: 14min 36s
Wall time: 18min 48s


## Cheaty evaluating on test labels

In [0]:
submission = pd.read_csv(submission_path)
submission[TARGET_COLS] = (pred)

In [0]:
submission.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998985,4.945387e-01,0.987257,0.097632,0.974627,0.440083
1,0000247867823ef7,0.007164,2.753735e-05,0.001133,0.000093,0.000974,0.000146
2,00013b17ad220c46,0.007884,1.225024e-04,0.002532,0.000309,0.002714,0.000617
3,00017563c3f7919a,0.000261,8.791685e-07,0.000062,0.000005,0.000044,0.000008
4,00017695ad8997eb,0.007049,2.688169e-05,0.001406,0.000199,0.000862,0.000122
5,0001ea8717f6de06,0.000263,5.960464e-07,0.000046,0.000005,0.000044,0.000006
6,00024115d4cbde0f,0.002175,3.531575e-06,0.000255,0.000021,0.000245,0.000028
7,000247e83dcc1211,0.594126,4.024178e-03,0.067739,0.007929,0.081680,0.004874
8,00025358d4737918,0.117647,1.052022e-04,0.007950,0.000294,0.020561,0.001072
9,00026d1092fe71cc,0.001731,6.213784e-06,0.000261,0.000031,0.000221,0.000039


In [0]:
# labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)
test_labels_df = pd.read_csv(test_labels_filepath)

In [0]:
test_labels_df = test_labels_df[(test_labels_df["toxic"] != -1) &
                                (test_labels_df["severe_toxic"] != -1) &
                                (test_labels_df["obscene"] != -1) &
                                (test_labels_df["threat"] != -1) &
                                (test_labels_df["insult"] != -1) &
                                (test_labels_df["identity_hate"] != -1)]
test_labels_df.shape                               

(63978, 7)

In [0]:
submission_to_evaluate = submission[submission['id'].isin(test_labels_df['id'].values)]
submission_to_evaluate.shape

(63978, 7)

In [0]:
scores = []
for class_name in TARGET_COLS:
    train_target = test_labels_df[class_name]
    train_predicted = submission_to_evaluate[class_name]

    cv_score = np.mean(roc_auc_score(train_target.values, train_predicted.values))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.967239853233377
CV score for class severe_toxic is 0.9892108184637406
CV score for class obscene is 0.9771303428007556
CV score for class threat is 0.991811197712763
CV score for class insult is 0.9751569740998813
CV score for class identity_hate is 0.9831161975458
Total CV score is 0.9806108973093862


In [0]:
submission.to_csv(os.path.join(data_folder,"submission_bidirectional_GRU.csv"), index = False)

Жирный шрифт означает изменения в процессе валидации и оценивания и при изменении действует начиная со строчки указания и ниже

glove twitter 200:  0.98052

**С поднятием колва epochs и global_epochs + RocAucEarlyStopping** скор упал с 0.98052 до 0.97573

GLOBAL_EPOCHS = 2, EPOCHS = 5, glove.840B.300d.txt   0.97994  
BPEmb(lang="en", dim=25, vs=20000)   0.97375  
BPEmb(lang="en", dim=300, vs=20000) without preprocessing   0.97699  
BPEmb(lang="en", dim=300, vs=20000)   0.97865   
fasttext crawl-300d-2M-subword 0.95732  
BPEmb(lang="en", dim=300, vs=20000) **Patience 2->1 SEEDS 1->1**  0.98041   
flair ELMoEmbeddings('small') embedding len 768  0.98005  
flair ELMoEmbeddings('medium') embedding len 1536  0.98038  
flair RoBERTaEmbeddings('roberta-base') embedding len 768  0.97413  
flair stack of ELMoEmbeddings('small') + RoBERTaEmbeddings('roberta-base') embedding len 1536  0.98069      
